In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px

from sklearn import linear_model
from sklearn import tree
from sklearn import ensemble
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [40]:

df = pd.read_csv("data/final_mat.csv")
df = df.drop('Unnamed: 0',axis=1)
df = df.drop('country',axis=1)
df["Дата"]= pd.to_datetime(df['Дата'],dayfirst=True)
df["liga"] = df["общая информация"].apply(lambda x: x.split(",")[0])
df["Tour"] = df["общая информация"].apply(lambda x: x.split(",")[1])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4328 entries, 0 to 4327
Data columns (total 73 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              4328 non-null   int64         
 1   общая информация                4328 non-null   object        
 2   Дата                            4328 non-null   datetime64[ns]
 3   Команда1                        4328 non-null   object        
 4   счет1                           4328 non-null   int64         
 5   Команда2                        4328 non-null   object        
 6   счет2                           4328 non-null   int64         
 7   Температура                     4328 non-null   int64         
 8   Признак погоды                  4328 non-null   object        
 9   поб1                            4328 non-null   float64       
 10  X                               4328 non-null   float64       
 11  поб2

In [49]:
bundes_l = df[df["liga"] == "Бундеслига"]
bundes_l[bundes_l["Дата"] > pd.to_datetime('2022-08-06 16:25:00')]

,id,общая информация,Дата,Команда1,счет1,Команда2,счет2,Температура,Признак погоды,поб1,...,Желтые карточки 2,Желтые карточки (соперник) 1,Желтые карточки (соперник) 2,Красные карточки 1,Красные карточки 2,Красные карточки (соперник) 1,Красные карточки (соперник) 2,тотал,liga,Tour
1,1736826,"Бундеслига, 1-й тур, регулярный сезон, 06.08.2...",2022-08-06 19:30:00,Боруссия Дортмунд,1,Байер,0,27,солнечно,2.13,...,0.89,1.25,1.00,0.0,0.00,0.00,0.0,1,Бундеслига,1-й тур
2,1736827,"Бундеслига, 1-й тур, регулярный сезон, 06.08.2...",2022-08-06 16:30:00,Унион Берлин,3,Герта,1,28,солнечно,1.87,...,1.75,0.80,0.75,0.0,0.25,0.00,0.0,4,Бундеслига,1-й тур
3,1736828,"Бундеслига, 1-й тур, регулярный сезон, 07.08.2...",2022-08-07 18:30:00,Кёльн,3,Шальке,1,30,солнечно,2.08,...,0.40,1.67,0.00,0.0,0.00,0.00,0.0,4,Бундеслига,1-й тур
4,1736829,"Бундеслига, 1-й тур, регулярный сезон, 06.08.2...",2022-08-06 16:30:00,Боруссия М,3,Хоффенхайм,1,27,солнечно,2.07,...,2.00,0.57,1.40,0.0,0.00,0.00,0.0,4,Бундеслига,1-й тур
5,1736830,"Бундеслига, 1-й тур, регулярный сезон, 06.08.2...",2022-08-06 16:30:00,Вольфсбург,2,Вердер,2,27,солнечно,2.11,...,0.20,1.80,0.00,0.0,0.00,0.00,0.0,4,Бундеслига,1-й тур
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,1737118,"Бундеслига, 33-й тур, регулярный сезон, 21.05....",2023-05-21 18:30:00,Аугсбург,0,Боруссия Дортмунд,3,28,переменная облачность,6.20,...,1.90,2.50,1.80,0.1,0.00,0.00,0.1,3,Бундеслига,33-й тур
292,1737119,"Бундеслига, 33-й тур, регулярный сезон, 20.05....",2023-05-20 16:30:00,Герта,1,Бохум,1,28,солнечно,2.36,...,2.00,1.56,2.30,0.0,0.00,0.11,0.1,2,Бундеслига,33-й тур
293,1737120,"Бундеслига, 33-й тур, регулярный сезон, 20.05....",2023-05-20 16:30:00,Шальке,2,Айнтрахт Франкфурт,2,28,солнечно,2.75,...,2.30,2.20,2.10,0.0,0.00,0.10,0.2,4,Бундеслига,33-й тур
294,1737121,"Бундеслига, 33-й тур, регулярный сезон, 20.05....",2023-05-20 16:30:00,Вердер,1,Кёльн,1,24,солнечно,2.46,...,2.56,2.10,2.33,0.0,0.00,0.00,0.0,2,Бундеслига,33-й тур


In [52]:
bundes_l["Команда2"].value_counts()

Кёльн                  81
Унион Берлин           80
Майнц                  79
Фрайбург               79
Вольфсбург             79
Боруссия М             78
Бавария                77
Хоффенхайм             77
Аугсбург               77
Байер                  76
Герта                  76
Айнтрахт Франкфурт     76
Боруссия Дортмунд      75
РБ Лейпциг             74
Вердер                 71
Шальке                 70
Штутгарт               60
Арминия Билефельд      44
Бохум                  25
Падерборн 07           17
Фортуна Дюссельдорф    17
Гройтер                10
Хольштайн Киль          2
Хайденхайм              1
Name: Команда2, dtype: int64

In [53]:
df["Tour"].value_counts()

 9-й тур                        134
 8-й тур                        134
 11-й тур                       134
 10-й тур                       134
 32-й тур                       125
 31-й тур                       125
 30-й тур                       125
 29-й тур                       125
 7-й тур                        124
 6-й тур                        124
 4-й тур                        124
 2-й тур                        124
 3-й тур                        123
 33-й тур                       123
 5-й тур                        122
 34-й тур                       117
 16-й тур                       116
 13-й тур                       116
 17-й тур                       116
 15-й тур                       116
 18-й тур                       116
 12-й тур                       116
 19-й тур                       116
 20-й тур                       116
 14-й тур                       115
 21-й тур                       113
 1-й тур                        110
 22-й тур                   

In [56]:
df = df.drop_duplicates()
df["Tour"].value_counts()

 9-й тур                        125
 8-й тур                        125
 11-й тур                       125
 10-й тур                       125
 32-й тур                       116
 31-й тур                       116
 30-й тур                       116
 29-й тур                       116
 7-й тур                        115
 6-й тур                        115
 4-й тур                        115
 2-й тур                        115
 3-й тур                        114
 33-й тур                       114
 5-й тур                        113
 34-й тур                       108
 16-й тур                       107
 13-й тур                       107
 17-й тур                       107
 15-й тур                       107
 18-й тур                       107
 12-й тур                       107
 19-й тур                       107
 20-й тур                       107
 14-й тур                       106
 21-й тур                       104
 1-й тур                        101
 22-й тур                   

In [55]:
df

,id,общая информация,Дата,Команда1,счет1,Команда2,счет2,Температура,Признак погоды,поб1,...,Желтые карточки 2,Желтые карточки (соперник) 1,Желтые карточки (соперник) 2,Красные карточки 1,Красные карточки 2,Красные карточки (соперник) 1,Красные карточки (соперник) 2,тотал,liga,Tour
0,1736825,"Бундеслига, 1-й тур, регулярный сезон, 05.08.2...",2022-08-05 21:35:00,Айнтрахт Франкфурт,1,Бавария,6,29,солнечно,6.50,...,1.90,1.00,1.50,0.00,0.10,0.25,0.00,7,Бундеслига,1-й тур
1,1736826,"Бундеслига, 1-й тур, регулярный сезон, 06.08.2...",2022-08-06 19:30:00,Боруссия Дортмунд,1,Байер,0,27,солнечно,2.13,...,0.89,1.25,1.00,0.00,0.00,0.00,0.00,1,Бундеслига,1-й тур
2,1736827,"Бундеслига, 1-й тур, регулярный сезон, 06.08.2...",2022-08-06 16:30:00,Унион Берлин,3,Герта,1,28,солнечно,1.87,...,1.75,0.80,0.75,0.00,0.25,0.00,0.00,4,Бундеслига,1-й тур
3,1736828,"Бундеслига, 1-й тур, регулярный сезон, 07.08.2...",2022-08-07 18:30:00,Кёльн,3,Шальке,1,30,солнечно,2.08,...,0.40,1.67,0.00,0.00,0.00,0.00,0.00,4,Бундеслига,1-й тур
4,1736829,"Бундеслига, 1-й тур, регулярный сезон, 06.08.2...",2022-08-06 16:30:00,Боруссия М,3,Хоффенхайм,1,27,солнечно,2.07,...,2.00,0.57,1.40,0.00,0.00,0.00,0.00,4,Бундеслига,1-й тур
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4323,15326853,"Серия А, 8-й тур, регулярный сезон, 16.10.2021...",2021-10-16 21:50:00,Милан,3,Верона,2,19,в отдельных районах умеренный или сильный дожд...,1.57,...,1.89,2.00,1.44,0.10,0.11,0.00,0.11,5,Серия А,8-й тур
4324,15326852,"Серия А, 8-й тур, регулярный сезон, 16.10.2021...",2021-10-16 19:00:00,Лацио,3,Интер,1,23,переменная облачность,3.28,...,1.70,1.90,2.20,0.10,0.00,0.20,0.10,4,Серия А,8-й тур
4325,15326855,"Серия А, 8-й тур, регулярный сезон, 16.10.2021...",2021-10-16 16:00:00,Специя,2,Салернитана,1,20,переменная облачность,2.13,...,2.11,2.10,2.22,0.20,0.11,0.00,0.22,3,Серия А,8-й тур
4326,15326837,"Серия А, 7-й тур, регулярный сезон, 03.10.2021...",2021-10-03 21:45:00,Аталанта,2,Милан,3,18,небольшой дождь,2.20,...,1.20,2.67,1.90,0.00,0.10,0.11,0.00,5,Серия А,7-й тур
